In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt


from konlpy.tag import Kkma
from eunjeon import Mecab

from tqdm.notebook import tqdm

from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, make_scorer
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_validate, StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [2]:
naverNews = pd.read_csv('data/naverNewsFinal.csv')

naverNews['감정표현'] = naverNews['감정표현'].apply(lambda x : [int(i) for i in x[1:-1].split(', ')])

naverNews.tail(2)

,시각,카테고리,미디어,제목,내용,url,수정시간,감정표현,timestamp
7987205,2020.12.31. 오전 12:00,사회,동아일보,경기도 공공배달앱 ‘배달특급’ 20여일만에 거래액 20억 돌파,낮은 수수료로 배달매출에 기댄 소상공인들의 부담 확 덜어줘 경기 오산시에서 중국집을...,https://news.naver.com/main/read.nhn?mode=LSD&...,NaN,"[4, 0, 0, 1, 0]",1.610066e+09
7987206,2020.12.31. 오전 12:00,사회,뉴시스,녹유 오늘의 운세51년생 침이 고여지는 대접을 받아요,서울 뉴시스 녹유 錄喩 의 오늘의 운세 2020년 12월 31일 목요일 음력 11월...,https://news.naver.com/main/read.nhn?mode=LSD&...,NaN,"[0, 0, 0, 0, 0]",1.610066e+09


In [3]:
mergeDF = pd.read_csv('final_dataset.csv')

mergeDF['감정표현'] = mergeDF['감정표현'].apply(lambda x : [int(i) for i in x[1:-1].split(', ')])
mergeDF['sum'] = mergeDF[['제보횟수','악의적헤드라인', '헛소리선동', '사실 및 통계왜곡']].sum(axis = 1)

mergeDF.tail(2)

FileNotFoundError: [Errno 2] No such file or directory: 'final_dataset.csv'

##### 데이터 셋 나누기
- 1. 진짜뉴스 
- 2. 가짜뉴스  (sum >= 3)

In [ ]:
print('sum값 0의 개수 : ', len(mergeDF[mergeDF['sum']<1]))
print('sum값 1이상 2이하 개수 : ', len(mergeDF[(mergeDF['sum']>=1)  & (mergeDF['sum']<9)]))
print('sum값 3이상 개수 : ', len(mergeDF[mergeDF['sum']>=9]))

neutral_news = mergeDF[(mergeDF['sum']>=1)  & (mergeDF['sum']<9)]
fake_news = mergeDF[mergeDF['sum']>=3]

print(neutral_news.shape, fake_news.shape)

##### 언론사별 특징 파악

In [ ]:
category_media_fake_cnt = mergeDF.groupby(['카테고리','미디어']).sum('사실 및 통계왜곡').sort_values(by = '사실 및 통계왜곡', ascending = False).reset_index()
category_media_fake_cnt.head(10)

In [ ]:
media_naver_total_cnt = pd.DataFrame(naverNews[['미디어','카테고리']].value_counts()).reset_index().rename(columns = {0 : 'total_news_cnt'})
media_naver_reportash_cnt = pd.DataFrame(mergeDF[['미디어','카테고리']].value_counts()).reset_index().rename(columns = {0 : 'reportash_cnt'})


category_media_fake_cnt = pd.merge(category_media_fake_cnt, media_naver_total_cnt, how = 'left')
category_media_fake_cnt = pd.merge(category_media_fake_cnt, media_naver_reportash_cnt, how = 'left')


reportrash_rate = category_media_fake_cnt['reportash_cnt'] / category_media_fake_cnt['total_news_cnt'].values
category_media_fake_cnt.loc[:, 'reportrash_rate(%)'] = reportrash_rate * 100
category_media_fake_cnt= category_media_fake_cnt.sort_values(by = 'reportrash_rate(%)', ascending = False)
category_media_fake_cnt= category_media_fake_cnt.drop('newsID', axis = 1 ).reset_index(drop = True)

category_media_fake_cnt.head(5)

In [ ]:
low_rate_polictics_media = list(category_media_fake_cnt[category_media_fake_cnt['카테고리'] == '정치'].tail(5)['미디어'])
low_rate_economy_media = list(category_media_fake_cnt[category_media_fake_cnt['카테고리'] == '경제'].tail(5)['미디어'])
low_rate_social_media = list(category_media_fake_cnt[category_media_fake_cnt['카테고리'] == '사회'].tail(5)['미디어'])

print(low_rate_polictics_media)
print(low_rate_economy_media)
print(low_rate_social_media)

In [ ]:
category_media_fake_cnt[category_media_fake_cnt['카테고리'] == '정치'].tail(5)

In [ ]:
category_media_fake_cnt[category_media_fake_cnt['카테고리'] == '경제'].tail(5)

In [ ]:
category_media_fake_cnt[category_media_fake_cnt['카테고리'] == '사회'].tail(5)

In [ ]:
naverNews['좋아요'] = naverNews['감정표현'].apply(lambda x : x[0])
naverNews['훈훈해요'] = naverNews['감정표현'].apply(lambda x : x[1])
naverNews['슬퍼요'] = naverNews['감정표현'].apply(lambda x : x[2])
naverNews['화나요'] = naverNews['감정표현'].apply(lambda x : x[3])
naverNews['후속기사원해요'] = naverNews['감정표현'].apply(lambda x : x[4])

naverNews['감정표현sum'] = naverNews[['좋아요','훈훈해요','슬퍼요','화나요','후속기사원해요']].sum(axis = 1)

naverNews.tail(2)

In [ ]:
mergeDF[mergeDF['감정표현'].apply(lambda x : sum(x)) < 10]

##### 진짜 정치뉴스 데이터의 후보군
- [정치, 경제, 사회] 3가지의 카테고리로 따로 후보군 데이터프레이 생성
    - 네이버뉴스 감정표현 합 10 이상 : 어느정도 이슈화된 뉴스를 선정 ex)날씨에 대한 뉴스, 주식 지수 관련뉴스 등 뉴스 제외
    - 카테고리별로 reportrash에 제보된 비율이 낮은 언론사들의 뉴스를 활용
    - 레포트래쉬에 제보된 뉴스는 제외

In [ ]:
naverNews.tail(2)

In [ ]:
#진짜 정치뉴스 데이터의 후보군
real_politics_news = naverNews[(naverNews['카테고리'] == '정치')  & \
                               (naverNews['미디어'].isin(low_rate_polictics_media)) &\
                               (naverNews['감정표현sum'] > 10) &\
                               (~naverNews['제목'].isin(mergeDF['제목']))]


#진짜 경제뉴스 데이터의 후보군
real_economy_news = naverNews[(naverNews['카테고리'] == '경제')  & \
                              (naverNews['미디어'].isin(low_rate_economy_media)) &\
                              (naverNews['감정표현sum'] > 10) &\
                              (~naverNews['제목'].isin(mergeDF['제목']))]


#진짜 사회뉴스 데이터의 후보군
real_social_news = naverNews[(naverNews['카테고리'] == '사회')  & \
                             (naverNews['미디어'].isin(low_rate_social_media)) &\
                             (naverNews['감정표현sum'] > 10) &\
                             (~naverNews['제목'].isin(mergeDF['제목']))]


real_news = pd.concat([real_politics_news, real_economy_news, real_social_news], ignore_index = True)


print(real_politics_news.shape, real_economy_news.shape, real_social_news.shape)

In [ ]:
import random

rand_index_11500 = random.sample(range(0, real_news.shape[0]), 11500)

real_news = real_news.iloc[rand_index_11500, :]
print(real_news.shape)

display(real_news.tail(2))

In [ ]:
real_news.loc[:, 'label'] = 0        #진짜 : 0
neutral_news.loc[:, 'label'] = 1     #중립 : 1
fake_news.loc[:, 'label'] = 2        #가짜 : 2

use_col_list = ['시각','카테고리','미디어','제목','내용','수정시간','감정표현', 'label']

finalDF = pd.concat([neutral_news[use_col_list], real_news[use_col_list], fake_news[use_col_list]], ignore_index = True)
finalDF['화나요rate'] = finalDF['감정표현'].apply(lambda x : x[3]/ (sum(x)+0.000001))
finalDF['수정시간'] = finalDF['수정시간'].apply(lambda x : 0 if x is np.nan else 1) # 수정 : 1 , 수정안했으면 0

finalDF

In [ ]:
finalDF.groupby(['카테고리', 'label']).describe()

In [ ]:
finalDF.groupby(['카테고리', 'label']).count()

In [ ]:
finalDF['label'].value_counts().sort_index()

In [ ]:
finalDF.head(50)

In [ ]:
finalDF['내용'].apply(lambda x : x[-30:]).head(50)

In [ ]:
finalDF.tail(2)

In [ ]:
# #한글자 단어 중, 유용한 단어거나, 동음이의어가 아닌 하나의 의미만 갖는 단어 추출
# import collections

# rawContent = finalDF['내용'].apply(lambda x : tokenizer.morphs(x))

# tokList = [i for i in rawContent]

# collections.Counter([x for sublist in tokList for x in sublist if len(x) == 1]).most_common(400)

In [ ]:
del naverNews, mergeDF

In [ ]:
finalDF.to_csv('data/useModelingData_label3.csv', index = False)

In [ ]:
# 가장 많이 있는 가짜뉴스 언론사는 ?
finalDF[finalDF['label'] == 2]['미디어'].value_counts()

In [ ]:
stop

## <center>최종 데이터셋을 활용한 Modeling</center>

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt


from konlpy.tag import Kkma
from eunjeon import Mecab

from tqdm.notebook import tqdm

from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, make_scorer
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_validate
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

from gensim.models.doc2vec import Doc2Vec, TaggedDocument

##### 1. 형태소분석기 활용

In [ ]:
#finalDF = pd.read_csv('useModelingData.csv')



In [ ]:
#년 월 데이터만 활용하기로,,
finalDF['시각'] = finalDF['시각'].str[:7].apply(lambda x : int(x.replace('.', '')))

# 감정표현 string => list 변환
finalDF['감정표현'] = finalDF['감정표현'].apply(lambda x : [int(i) for i in x[1:-1].split(', ')])

In [ ]:
finalDF['좋아요'] = finalDF['감정표현'].apply(lambda x : x[0])
finalDF['훈훈해요'] = finalDF['감정표현'].apply(lambda x : x[1])
finalDF['슬퍼요'] = finalDF['감정표현'].apply(lambda x : x[2])
finalDF['화나요'] = finalDF['감정표현'].apply(lambda x : x[3])
finalDF['후속기사원해요'] = finalDF['감정표현'].apply(lambda x : x[4])

finalDF.tail(2)

In [ ]:
import re
import string

string_punctuation = '!"#$%&\'()*+,-./:;<=>?[\\]^_`{|}~…’'

re_tok = re.compile(f'([{string_punctuation}])')


In [ ]:
#tokenizer = Okt()
tokenizer = Mecab()
#tokenizer = Okt()

re_tok = re.compile(f'([{string_punctuation}])')   # @ 제외한  특수문자 제거

token_title_list = []
token_content_list = []



# 자주 등장하며, 불용어가 아닌 1글자 단어 List
useLen1WordList = [
                '년','월','주','문','군','적','핵','명','조','억','또','법','측','뒤','힘','앉',
                '글','열','신','률','왔','찾','놓','못','않','없','때','많','딸','뜻','왜','끝',
                '곧','액','꿈','뽑','촌','잃','밖','믿','탈','피','편','쉽','첫','본','박','꾼',
                   
                '文','與','野','美','日','韓','中','靑','北','軍','無','前',
                '檢','反','法','黃','秋','發','康','南','朴','男','女','尹','故']
 


finalDF['제목'] = finalDF['제목'].apply(lambda title : re_tok.sub(r'', title))
finalDF['내용'] = finalDF['내용'].apply(lambda content : re_tok.sub(r'', content))


for text in tqdm(finalDF['제목']):
    
    token = tokenizer.morphs(text)
    token = [t for t in token if (len(t) >= 2) | (t in useLen1WordList)]
    token_title_list.append(token)

    
for text in tqdm(finalDF['내용']):
    text = re_tok.sub(r'', text)
    
    token = tokenizer.morphs(text)
    token = [t for t in token if (len(t) >= 2) | (t in useLen1WordList)]
    token_content_list.append(token)

##### Solution1. Doc2vec

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
#제목 + 내용
total_token = [title + content for title, content in zip(token_title_list, token_content_list)]

In [ ]:
tagged_data  = [TaggedDocument(d, [i]) for i, d in enumerate(total_token)]

In [ ]:
# 8일때도 좋게나옴

doc2vec_model = Doc2Vec(documents = tagged_data,
             vector_size = 300, 
             window = 7, 
             min_count = 2,
             alpha = 0.0025)

In [ ]:
X_text = doc2vec_model.docvecs.vectors_docs

In [ ]:
from sklearn.preprocessing import LabelEncoder

le_media = LabelEncoder()
le_media.fit(finalDF['미디어'])
finalDF['미디어'] = le_media.transform(finalDF['미디어'])

le_category = LabelEncoder()
le_category.fit(finalDF['카테고리'])
finalDF['카테고리'] = le_category.transform(finalDF['카테고리'])

In [ ]:
finalDF

In [ ]:
#X_values = finalDF[['시각','수정시간','화나요rate','카테고리_경제', '카테고리_사회', '카테고리_정치']].values
X_values = finalDF[['시각','수정시간','화나요rate',
                    '좋아요','훈훈해요','슬퍼요','화나요','후속기사원해요','미디어']].values


X = np.hstack((X_text, X_values))
X = pd.DataFrame(X)   # 추후에,  test 데이터의 index 뽑아내기 위해 데이터프레임으로 변경함

y = finalDF['label'].values

In [ ]:
# 삭제

# '좋아요',
# '훈훈해요',
# '슬퍼요',
# '화나요',
# '후속기사원해요'

##### train_test_split 

In [ ]:
X_train = X.sample(frac=0.9)
y_train = y[X_train.index]

X_test = X.drop(X_train.index)
y_test = y[X_test.index]

print(y_train.shape, y_test.shape)

##### <center>modeling</center>

##### model Scoring

In [ ]:
def model_scoring(model, X_test, y_test):
    predict = model.predict(X_test)
    print('\n\n#####classification_report#####')
    print(classification_report(y_test, predict))
    print('\n\n#####confusion Matrix#####')
    print(confusion_matrix(y_test, predict))
    display(pd.DataFrame(confusion_matrix(y_test, predict)))
    print('\n\n정확도 : ', accuracy_score(y_test, predict))
    
    
    

##### lgbm

In [ ]:
X_train

In [ ]:
import lightgbm

clf_lgbm = lightgbm.LGBMClassifier()

clf_lgbm.fit(X_train, y_train)

model_scoring(clf_lgbm, X_test, y_test)

In [ ]:
finalDF.loc[X_test[np.array([i[2] for i in clf_lgbm.predict_proba(X_test)]) > 0.8].index, : ].sample(frac = 0.2)

##### kfold lightgbm

In [ ]:
def printKfoldAcc(model, k = 8):
    

    print ('#### model : {} ####'.format(model))
    kfold  = KFold(n_splits = k, shuffle = True)
    cv_accuracy=[]

    n_iter = 0

    # KFold 객체의 split()을 호출하면 폴드 별 학습용, 검증용 테스트의 로우 인덱스를 array로 반환
    for train_index, test_index in kfold.split(X.values):
        # kfold.split()으로 반환된 인덱스를 이용해 학습용, 검증용 테스트 데이터 추출
        X_train, X_test = X.loc[train_index, :], X.loc[test_index, :]
        y_train, y_test = y[train_index], y[test_index]

        # 학습 및 예측

        clf_lgbm.fit(X_train,y_train)
        pred = clf_lgbm.predict(X_test)
        n_iter += 1

        # 반복 시마다 정확도 측정
        accuracy = np.round(accuracy_score(y_test, pred), 4) * 100
        train_size = X_train.shape[0]
        test_size = X_test.shape[0]

        print('#{} 검증 정확도 : {} %'.format(n_iter, accuracy))
        cv_accuracy.append(accuracy)

    print('\n평균 정확도:{:.2f} % '.format(np.mean(cv_accuracy)))
    model_scoring(clf_lgbm, X_test, y_test)
    
printKfoldAcc(model = clf_lgbm, k = 8)

#model_scoring(clf_lgbm, X_test, y_test)

In [ ]:
testing_data = finalDF.loc[X_test.index, :]

testing_data['미디어'] = le_media.inverse_transform(testing_data['미디어'])
testing_data

In [ ]:
finalDF[finalDF['label'] == 2]['미디어'].value_counts()

In [ ]:
le_me

In [ ]:
le_media.inverse_transform([1, 2, 3])

In [ ]:
testing_data[testing_data['미디어'] == '조선일보']

In [ ]:
testing_data[testing_data['미디어'] == 25]

In [ ]:
print(le_media.inverse_transform([25]))

clf_lgbm.predict(testing_data[testing_data['미디어'] == '조선일보'].index, :])

In [ ]:
testing_data[testing_data['미디어'] == '조선일보'].tail(2)

In [ ]:
clf_lgbm.predict(X_test.loc[testing_data[testing_data['미디어'] == '조선일보'].index, :])

In [ ]:
clf_lgbm.predict(X_test.loc[testing_data[testing_data['미디어'] == '뉴시스'].index, :])

In [ ]:
finalDF[finalDF['label'] == 1]['미디어'].value_counts()

In [ ]:
finalDF[finalDF['label'] == 2]['미디어'].value_counts()

In [ ]:
X_test[np.array([i[2] for i in clf_lgbm.predict_proba(X_test)]) > 0.8].index

In [ ]:
finalDF.loc[X_test[np.array([i[2] for i in clf_lgbm.predict_proba(X_test)]) > 0.8].index, :].sample(frac = 0.2)

In [ ]:
# 69 % : 화나요rate / 아무것도없이 초간단 전처리

# 72 % : 화나요rate / 한글자 단어들 추가 (문 , 군, 적, 핵, 법, '년','월','주','문','군','적','핵','명','조','억','또','법','측','뒤','힘','앉',)

# 75 % : 화나요rate / 모든감정표현5개 / 한글자 단어들 추가


##### 삭제가능 ~

In [ ]:
clf_rf = RandomForestClassifier()
# clf_svc = SVC(kernel='linear', C=1)
# clf_NB = GaussianNB()


clf_rf.fit(X_train, y_train)
# clf_svc.fit(X_train, y_train)
# clf_NB.fit(X_train, y_train)

In [ ]:
model_scoring(clf_rf, X_test, y_test)

In [ ]:
# test셋을 통해서 결과 확인
threshold = 0.01
test_predict_fake_index = np.where(np.array([i[2] for i in clf_rf.predict_proba(X_test)]) > threshold)[0]

finalDF.loc[test_predict_fake_index, :].tail(30)

In [ ]:
threshold = 0.01
train_predict_fake_index = np.where(np.array([i[2] for i in clf_rf.predict_proba(X_train)]) > threshold)[0]

finalDF.loc[train_predict_fake_index, :].tail(30)

In [ ]:
finalDF['label'].value_counts()

In [ ]:
[i for i in clf_rf.predict_proba(X_test)]

##### svc

In [ ]:
clf_svc = SVC(kernel='linear', C=1)

clf_svc.fit(X_train.loc[:, :299], y_train)

In [ ]:
model_scoring(clf_svc, X_test, y_test)

##### naive bayies

In [ ]:
clf_NB = GaussianNB()

clf_NB.fit(X_train.loc[:, :299], y_train)

model_scoring(clf_NB, X_test.loc[:, :299], y_test)

In [ ]:
finalDF[finalDF['label'] == 0]